# Amazon SageMaker で LightGBM を使う

このノートブックでは、LightGBM のカスタムコンテナを作成し、それを使って Amazon SageMaker でモデルを学習、デプロイします。カスタムコンテナを作る方法や、作ったコンテナを使ってモデルを学習する方法の参考になればと思います。また、Amazon SageMaker Experiments を使って複数の学習ジョブのメトリクスを比較する方法も紹介しています。

このノートブックは、[こちら](https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/lightgbm-byo/lightgbm-byo.ipynb) を参考に作成しました。使用するデータセットは [こちら](https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset) で、オンラインショッピングの顧客が商品を購入するかどうかを予測するモデルを作成します。

## 準備

In [ ]:
%matplotlib inline

import boto3
import sys
import sagemaker
import numpy as np
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
session = sagemaker.Session()
s3_output = session.default_bucket()
s3_prefix = 'lightGBM-BYO'

このノートブックでは Amazon SageMaker Experiments を利用するため、必要なライブラリをインストールします。

In [ ]:
!{sys.executable} -m pip install sagemaker-experiments requests

In [ ]:
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
import time

lightgbm_experiment = Experiment.create(
    experiment_name=f"lightgbm-{int(time.time())}", 
    description="Purchase intent prediction with lightGBM", 
    sagemaker_boto_client=boto3.client('sagemaker'))
print(lightgbm_experiment)

このサンプルでは、カスタムコンテナを Amazon ECR に push する必要があります。以下の操作でこのノートブックインスタンスで使用している IAM ロールに Amazon ECR にイメージを push するための権限を追加してください。

1. Amazon SageMaker コンソールからこのノートブックインスタンスの詳細画面を表示<br>（左側のメニューのインスタンス -> ノートブックインスタンス -> インスタンス名をクリック）
1. 「アクセス許可と暗号化」の「IAM ロール ARN」のリンクをクリック（IAM のコンソールに遷移します）
1. 「ポリシーをアタッチします」と書いてある青いボタンをクリック
1. 検索ボックスに ec2containerregistry と入力し  AmazonEC2ContainerRegistryFullAccess のチェックボックスをチェックする
1. 「ポリシーのアタッチ」と書いてある青いボタンをクリック

In [ ]:
!mkdir -p raw
!wget -P ./raw https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv

In [ ]:
import pandas as pd

df = pd.read_csv('./raw/online_shoppers_intention.csv')
df.head()

In [ ]:
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

sns.countplot(x=df['Revenue'])
plt.ylim(0,12000)
plt.xlabel('Transactions Completed', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.text(x=-.175, y=11000 ,s='10,422', fontsize=16)
plt.text(x=.875, y=2500, s='1908', fontsize=16)
plt.show()

ダウンロードしたデータを Amazon S3 にアップロードします。

In [ ]:
rawdata_s3_prefix = '{}/raw'.format(s3_prefix)
raw_s3 = session.upload_data(path='./raw/', key_prefix=rawdata_s3_prefix)
print(raw_s3)

## Amazon SageMaker Processing でデータの前処理

Amazon SageMaker Processing は、前処理などをジョブとして実行する際に便利です。任意のコンテナとスクリプトを指定することで、様々な処理をジョブとして実行することができます。ジョブの情報は Amazon SageMaker によって記録・管理されます。

In [ ]:
!mkdir -p docker-proc-evaluate

In [ ]:
%%writefile docker-proc-evaluate/Dockerfile

FROM python:3.7-slim-buster
RUN apt -y update && apt install -y --no-install-recommends \
    libgomp1 \
    && apt clean    
RUN pip3 install lightgbm numpy pandas scikit-learn 
ENV PYTHONUNBUFFERED=TRUE
ENTRYPOINT ["python3"]

In [ ]:
ecr_repository = 'lightgbm-byo-proc-eval'
tag = ':latest'
uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker-proc-evaluate
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

In [ ]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=processing_repository_uri,
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge')

In [ ]:
%%writefile preprocessing.py

import glob
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split


if __name__=='__main__':
    
    input_file = glob.glob('{}/*.csv'.format('/opt/ml/processing/input'))
    print('\nINPUT FILE: \n{}\n'.format(input_file))   
    df = pd.read_csv(input_file[0])
    
    # minor preprocessing (drop some uninformative columns etc.)
    print('Preprocessing the dataset . . . .')   
    df_clean = df.drop(['Month','Browser','OperatingSystems','Region','TrafficType','Weekend'], axis=1)
    visitor_encoded = pd.get_dummies(df_clean['VisitorType'], prefix='Visitor_Type', drop_first = True)
    df_clean_merged = pd.concat([df_clean, visitor_encoded], axis=1).drop(['VisitorType'], axis=1)
    X = df_clean_merged.drop('Revenue', axis=1)
    y = df_clean_merged['Revenue']
    
    # split the preprocessed data with stratified sampling for class imbalance
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=2, test_size=.2)

    # save to container directory for uploading to S3
    print('Saving the preprocessed dataset . . . .')   
    train_data_output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
    np.save(train_data_output_path, X_train.to_numpy())
    train_labels_output_path = os.path.join('/opt/ml/processing/train', 'y_train.npy')
    np.save(train_labels_output_path, y_train.to_numpy())    
    test_data_output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
    np.save(test_data_output_path, X_test.to_numpy())
    test_labels_output_path = os.path.join('/opt/ml/processing/test', 'y_test.npy')
    np.save(test_labels_output_path, y_test.to_numpy())

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "lightgbm-byo-process-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(s3_output, s3_prefix)

script_processor.run(code='preprocessing.py',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                        source=raw_s3,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train',
                                                destination='{}/train'.format(output_destination),
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test',
                                                destination='{}/test'.format(output_destination),
                                                source='/opt/ml/processing/test')]
                    )

preprocessing_job_description = script_processor.jobs[-1].describe()

In [ ]:
output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'train':
        preprocessed_training_data = output['S3Output']['S3Uri']
        print(preprocessed_training_data)
    if output['OutputName'] == 'test':
        preprocessed_test_data = output['S3Output']['S3Uri']
        print(preprocessed_test_data)

あとでモデルの評価をノートブックインスタンス上でするために、前処理済みのデータをノートブックインスタンスにダウンロードしておきます。

In [ ]:
session.download_data(path='.', bucket=s3_output, key_prefix=s3_prefix+'/data/test/x_test.npy')
session.download_data(path='.', bucket=s3_output, key_prefix=s3_prefix+'/data/test/y_test.npy')

## モデルの学習

In [ ]:
!mkdir -p docker-train

argparse を使ってハイパーパラメタの設定を受け取ってモデルの学習で使用します。学習したモデルは、Estimator から受け取った args.model_dir に保存します。ここに学習済みモデルを保存しておくと、Amazon SageMaker が自動的に Amazon S3 にアップロードします。

In [ ]:
%%writefile docker-train/train.py

import argparse
import glob
import lightgbm as lgb
import numpy as np
import os


if __name__=='__main__':
    
    # extract training data S3 location and hyperparameter values
    parser = argparse.ArgumentParser()
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--validation', type=str, default=os.environ['SM_CHANNEL_VALIDATION'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--num_leaves', type=int, default=28)
    parser.add_argument('--max_depth', type=int, default=5)
    parser.add_argument('--learning_rate', type=float, default=0.1)
    args = parser.parse_args()
    
    print('Loading training data from {}\n'.format(args.train))
    input_files = glob.glob('{}/*.npy'.format(args.train))
    print('\nTRAINING INPUT FILE LIST: \n{}\n'.format(input_files)) 
    for file in input_files:
        if 'x_' in file:
            x_train = np.load(file)
        else:
            y_train = np.load(file)      
    print('\nx_train shape: \n{}\n'.format(x_train.shape))
    print('\ny_train shape: \n{}\n'.format(y_train.shape))
    train_data = lgb.Dataset(x_train, label=y_train)
    
    print('Loading validation data from {}\n'.format(args.validation))
    eval_input_files = glob.glob('{}/*.npy'.format(args.validation))
    print('\nVALIDATION INPUT FILE LIST: \n{}\n'.format(eval_input_files)) 
    for file in eval_input_files:
        if 'x_' in file:
            x_val = np.load(file)
        else:
            y_val = np.load(file)      
    print('\nx_val shape: \n{}\n'.format(x_val.shape))
    print('\ny_val shape: \n{}\n'.format(y_val.shape))
    eval_data = lgb.Dataset(x_val, label=y_val)
    
    print('Training model with hyperparameters:\n\t num_leaves: {}\n\t max_depth: {}\n\t learning_rate: {}\n'
          .format(args.num_leaves, args.max_depth, args.learning_rate))
    parameters = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'is_unbalance': 'true',
        'boosting': 'gbdt',
        'num_leaves': args.num_leaves,
        'max_depth': args.max_depth,
        'learning_rate': args.learning_rate,
        'verbose': 1
    }
    num_round = 10
    bst = lgb.train(parameters, train_data, num_round, eval_data, verbose_eval=1)
    
    print('Saving model . . . .')
    bst.save_model(os.path.join(args.model_dir,'online_shoppers_model.txt'))

In [ ]:
%%writefile docker-train/Dockerfile

FROM python:3.7-slim-buster
RUN apt -y update && apt install -y --no-install-recommends \
    libgomp1 build-essential \
    && apt clean    
RUN pip install lightgbm numpy pandas scikit-learn sagemaker-training
COPY train.py /train.py
ENV SAGEMAKER_PROGRAM /train.py
ENV PYTHONUNBUFFERED=TRUE

In [ ]:
ecr_repository_train = 'lightgbm-byo-train'
uri_suffix = 'amazonaws.com'
train_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_train + tag)

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository_train docker-train
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_train
!docker tag {ecr_repository_train + tag} $train_repository_uri
!docker push $train_repository_uri

まずはローカルモードでモデルを学習します。ローカルモード使うと、ローカル（今回はノートブックインスタンス）で擬似的に学習ジョブを実行することができます。学習用インスタンスを別途起動する必要がないため、すぐに学習ジョブを実行できるのが特徴です。ローカルモードを使用するには、Estimator の引数に `instance_type='local'` を設定するだけです。

In [ ]:
from sagemaker.estimator import Estimator
import json

def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({
    'num_leaves': 32,
    'max_depth': 3,
    'learning_rate': 0.08})

estimator = Estimator(image_uri=train_repository_uri,
                      role=role,
                      instance_count=1,
                      instance_type='local', # training job will run locally
                      hyperparameters=hyperparameters,
                      base_job_name='lightgbm-byo')

In [ ]:
estimator.fit({'train': preprocessed_training_data, 'validation': preprocessed_test_data})

学習したモデルは Amaozn S3 に model.tar.gz として保存されます。ダウンロードして展開することで学習済みモデルを取得できます。

In [ ]:
!aws s3 cp {estimator.model_data} ./model/model.tar.gz
!tar -xvzf ./model/model.tar.gz -C ./model

## ノートブック上でモデルの評価

先ほどダウンロードした学習済みモデルを使ってモデルの評価を行なってみます。

In [ ]:
!pip install lightgbm

In [ ]:
import glob
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score

input_files = glob.glob('./*.npy')
for file in input_files:
    if 'x_' in file:
        x_test = np.load(file)
    else:
        y_test = np.load(file)

def eval(model_path):
            
    bst_loaded = lgb.Booster(model_file=model_path+'online_shoppers_model.txt')
    y_pred = bst_loaded.predict(x_test)
    
#     print('Evaluating model . . . .\n')    
    acc = accuracy_score(y_test.astype(int), y_pred.round(0).astype(int))
    auc = roc_auc_score(y_test, y_pred)
    print('==== result ====')
    print(model_path)
    print('Accuracy:  {:.2f}'.format(acc))
    print('AUC Score: {:.2f}'.format(auc))
    
eval('./model/')

問題なく学習されていそうなことが確認できました。

## インスタンスを使ってハイパーパラメタチューニング

作成したコンテナやスクリプトの動作確認ができたので、学習インスタンスを使ってハイパーパラメタチューニングをやってみましょう。ここでは、Amazon SageMaker Experiments の機能をも使って、各学習ジョブの結果を関連づけて記録し、あとで結果を比較します。

Estimator の引数に metric_definitions を設定することで、メトリクスの値を取得することができます。ここでは validation loss を取得するよう設定しています。

In [ ]:
trial_name_map = {}

for i, max_depth in enumerate([3, 6, 9, 12]):
    # create trial
    trial_name = f"lightgbm-training-depth-{max_depth}-{int(time.time())}"
    trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=lightgbm_experiment.experiment_name,
        sagemaker_boto_client=boto3.client('sagemaker'),
    )
    trial_name_map[max_depth] = trial_name
    
    hyperparameters = json_encode_hyperparameters({ 
                                                    'num_leaves': 32,
                                                    'max_depth': max_depth,
                                                    'learning_rate': 0.08 })

    estimator = Estimator(image_uri=train_repository_uri,
                          role=role,
                          instance_count=1,
                          instance_type='ml.c5.xlarge',
                          hyperparameters=hyperparameters,
                          enable_sagemaker_metrics=True,
                          metric_definitions=[
                            {'Name':'validation:loss', 'Regex':'.*loss: ([0-9\\.]+)'}
                           ]
                          )
    
    training_job_name = f"lightgbm-training-depth-{max_depth}-{int(time.time())}"
    # Now associate the estimator with the Experiment and Trial
    estimator.fit(
        inputs={'train': preprocessed_training_data, 'validation': preprocessed_test_data}, 
        job_name=training_job_name,
        experiment_config={
            "TrialName": trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        wait=False,
    )

## チューニング結果の確認

**以降のセルは、上記セルで実行した全ての学習ジョブが完了してから実行してください。**学習ジョブの状況は、Amazon SageMaker のコンソールの左側のメニューから トレーニング -> トレーニングジョブ とクリックすると確認できます。全てジョブが Completed になっていることを確認してください。

以下では、Amazon SageMaker Experiments を使って記録された各学習ジョブの情報を確認しています。Experiments にはたくさんの情報が記録されているため、以下ではパラメタ情報は max_depth に絞って表示しています。

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [ ]:
from sagemaker.session import Session

sess = boto3.Session()
sm = sess.client('sagemaker')

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=lightgbm_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.validation:loss.Min",
    sort_order="Ascending",
    parameter_names=['max_depth']
)

In [ ]:
trial_component_analytics.dataframe()

今回は、`metrics.validation:loss.Min` が最も小さいモデルを採用します。

In [ ]:
model_artifact = trial_component_analytics.dataframe().loc[0]['SageMaker.ModelArtifact - Value']
print(model_artifact)

In [ ]:
!aws s3 cp $model_artifact ./model2/model.tar.gz
!tar -xvzf ./model2/model.tar.gz -C ./model2

ハイパーパラメタチューニングの前後でモデルの性能は変わったでしょうか。

In [ ]:
eval('./model/')
eval('./model2/')

## モデルのデプロイと推論

ハイパーパラメタチューニングで最も Loss が小さかったモデルをデプロイして推論します。

In [ ]:
# create directory for inference sources
!mkdir -p docker-inference

In [ ]:
%%writefile docker-inference/model_script.py

from collections import namedtuple
import glob
import json
import logging
import os
import re

import lightgbm as lgb
import numpy as np
from sagemaker_inference import content_types, encoder

NUM_FEATURES = 12

class ModelHandler(object):
    """
    A lightGBM Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.model = None

    def initialize(self, context):
        """
        Initialize model. This will be called during model loading time
        :param context: Initial context contains model server system properties.
        :return: None
        """
        self.initialized = True
        properties = context.system_properties
        model_dir = properties.get("model_dir") 
        self.model = lgb.Booster(model_file=os.path.join(model_dir,'online_shoppers_model.txt'))
       

    def preprocess(self, request):
        """
        Transform raw input into model input data.
        :param request: list of raw requests
        :return: list of preprocessed model input data
        """        
        payload = request[0]['body']
        data = np.frombuffer(payload, dtype=np.float64)
        data = data.reshape((data.size // NUM_FEATURES, NUM_FEATURES))
        return data

    def inference(self, model_input):
        """
        Internal inference methods
        :param model_input: transformed model input data list
        :return: list of inference output in numpy array
        """
        prediction = self.model.predict(model_input)
        return prediction

    def postprocess(self, inference_output):
        """
        Post processing step - converts predictions to str
        :param inference_output: predictions as numpy
        :return: list of inference output as string
        """

        return [str(inference_output.tolist())]
        
    def handle(self, data, context):
        """
        Call preprocess, inference and post-process functions
        :param data: input data
        :param context: mms context
        """
        
        model_input = self.preprocess(data)
        model_out = self.inference(model_input)
        return self.postprocess(model_out)

_service = ModelHandler()


def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)

In [ ]:
%%writefile docker-inference/dockerd-entrypoint.py

import subprocess
import sys
import shlex
import os
from retrying import retry
from subprocess import CalledProcessError
from sagemaker_inference import model_server

def _retry_if_error(exception):
    return isinstance(exception, CalledProcessError or OSError)

@retry(stop_max_delay=1000 * 50,
       retry_on_exception=_retry_if_error)
def _start_mms():
    # by default the number of workers per model is 1, but we can configure it through the
    # environment variable below if desired.
    # os.environ['SAGEMAKER_MODEL_SERVER_WORKERS'] = '2'
    model_server.start_model_server(handler_service='/home/model-server/model_script.py:handle')

def main():
    if sys.argv[1] == 'serve':
        _start_mms()
    else:
        subprocess.check_call(shlex.split(' '.join(sys.argv[1:])))

    # prevent docker exit
    subprocess.call(['tail', '-f', '/dev/null'])
    
main()

In [ ]:
%%writefile docker-inference/Dockerfile

FROM ubuntu:18.04
    
# Set a docker label to advertise multi-model support on the container
LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

# Install necessary dependencies for MMS and SageMaker Inference Toolkit
RUN apt-get update && \
    apt-get -y install --no-install-recommends \
    build-essential \
    ca-certificates \
    openjdk-8-jdk-headless \
    python3-dev \
    curl \
    vim \
    && rm -rf /var/lib/apt/lists/* \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py

RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
    
RUN pip install lightgbm numpy pandas \ 
                scikit-learn multi-model-server \
                sagemaker-inference retrying

# Copy entrypoint script to the image
COPY dockerd-entrypoint.py /usr/local/bin/dockerd-entrypoint.py
RUN chmod +x /usr/local/bin/dockerd-entrypoint.py

RUN mkdir -p /home/model-server/

# Copy the default custom service file to handle incoming data and inference requests
COPY model_script.py /home/model-server/model_script.py

# Define an entrypoint script for the docker image
ENTRYPOINT ["python", "/usr/local/bin/dockerd-entrypoint.py"]

# Define command to be passed to the entrypoint
CMD ["serve"]

In [ ]:
# 以下のセルを実行して no space left のエラーが出たら以下のコマンドのコメントアウトを外して実行してください。
# !docker system prune -a -f

In [ ]:
ecr_repository_inference = 'lightgbm-byo-inference'
uri_suffix = 'amazonaws.com'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_inference + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_inference docker-inference
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_inference
!docker tag {ecr_repository_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

In [ ]:
from sagemaker import Model, Predictor

lgbm_model = Model(model_data=model_artifact,
                   role=role,
                   image_uri=inference_repository_uri,
                   name='lightgbm-byo-deployment')

lgbm_model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge', wait=True)

In [ ]:
from sklearn.metrics import confusion_matrix

predictor = Predictor(endpoint_name=lgbm_model.endpoint_name)
payload = x_test.tobytes()

response = predictor.predict(payload)

y_pred_str = response.decode('utf-8')
y_pred = np.asarray(y_pred_str.strip('][').split(', '), dtype=np.float64)
y_pred = (y_pred > 0.5).astype(int)

confusion_matrix(y_pred, y_test)

## リソースの削除

利用が終わったら、このノートブックを実行したノートブックインスタンスの停止および削除を実施してください。ノートブックインスタンスを停止させると、ノートブックインスタンスの課金は止まりますがアタッチされている EBS ボリュームへの課金が継続しますので、完全に課金を止めるにはノートブックインスタンスの停止だけでなく削除まで実施してください。

また、Amazon S3 にアップロードした各種ファイルに対しても課金が発生するため、不要であれば削除してください。

In [ ]:
predictor.delete_endpoint()

作成した Amazon SageMaker Experiments の情報も削除します。

In [ ]:
def cleanup(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                sagemaker_boto_client=sm,
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
    experiment.delete()

In [ ]:
cleanup(lightgbm_experiment)